In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
import random
import gc
               
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



In [2]:
fy_10 = pd.read_csv('fy_10_augmented.csv')
fy_11 = pd.read_csv('fy_11_augmented.csv')
fy_12 = pd.read_csv('fy_12_augmented.csv')
fy_13 = pd.read_csv('fy_13_augmented.csv')
fy_14 = pd.read_csv('fy_14_augmented.csv')
fy_15 = pd.read_csv('fy_15_augmented.csv')
fy_16 = pd.read_csv('fy_16_augmented.csv')


In [3]:
essentials = ['Unnamed: 0',
 'Fund Id',
 'Business Area',
 'Fund Center Id',
 'GL Account',
 'GL Description',
 'GL Category',
 'GL Category Description',
 'Revenue or Expenditure',
 'Fiscal Year',
 'Fund Name',
 'Fund Type Descr',
 'Business Area Name',
 'Fund Center Name',
 'Commitment Set Description',
 'Original Budget',
 'Current Budget',
 'Actuals',
 '3Y_Past_Actuals',
 '2Y_Past_Actuals',
 '1Y_Past_Actuals',
 '2Y_1Y_Actuals_Mean',
 '3Y_2Y_1Y_Actuals_Mean',
 '3Y_Past_Original_Budget',
 '2Y_Past_Original_Budget',
 '1Y_Past_Original_Budget',
 '2Y_1Y_Original_Budget_Mean',
 '3Y_2Y_1Y_Original_Budget_Mean',
 '3Y_Past_OA_Difference',
 '2Y_Past_OA_Difference',
 '1Y_Past_OA_Difference',
 '2Y_1Y_OA_Difference_Mean',
 '3Y_2Y_1Y_OA_Difference_Mean',
 'OA_Difference',
 'Yearly Crime Index',
 'Yearly Avg High Temp',
 'Yearly Avg Low Temp',
 'Yearly Rain Fall',
 'Astros Playoff Rounds',
 'Astros Yearly Attendance',
 'Rockets Playoff Rounds',
 'Texans Playoff Rounds',
 'Texans Average Attendance',
 'Rockets Total Attendance']

dummies = ['Fund Name_Asset Forfeiture Fund - Justice',
 'Fund Name_Asset Forfeiture Fund - State',
 'Fund Name_Auto Dealers Fund',
 'Fund Name_Building Inspection Fund',
 'Fund Name_C&E-Civic Center Facility Revenue Fund',
 'Fund Name_Cable Television Fund',
 'Fund Name_Child Safety Fund',
 'Fund Name_Digital Houston Special Revenue',
 'Fund Name_Expansion of Recycling Program',
 'Fund Name_Fleet Management Fund',
 'Fund Name_GSD - In-House Renovation Revolving Fund',
 'Fund Name_General Fund',
 'Fund Name_Greater Houston Trans & Emer Center',
 'Fund Name_HAS-Revenue Fund',
 'Fund Name_Health Benefits Fund',
 'Fund Name_Historic Preservation Fund',
 'Fund Name_Houston Emergeny Center',
 'Fund Name_Long-Term Disablility Plan Fund',
 'Fund Name_Municipal Court Technology Fund',
 'Fund Name_PWE-Combined Utility System Gen Pur Fund',
 'Fund Name_PWE-Combined Utility System Oper Fund',
 'Fund Name_PWE-W & S System Operating Fund',
 'Fund Name_Park Special Revenue Fund',
 'Fund Name_Parking Mangement Operating Fund',
 'Fund Name_Police Special Services',
 'Fund Name_Project Cost Recovery',
 'Fund Name_Property & Casualty Fund',
 'Fund Name_Stormwater Fund',
 'Fund Name_Supplemental Environmental Proj',
 'Fund Name_Workers Compensation Admin Fund',
 'Fund Type Descr_Enterprise Funds',
 'Fund Type Descr_General Funds',
 'Fund Type Descr_Internal Services Funds',
 'Fund Type Descr_Special Revenue Funds',
 'Business Area Name_Admn. & Regulatory Affairs',
 'Business Area Name_Business Opportunity',
 "Business Area Name_City Controller's Office",
 'Business Area Name_City Council',
 'Business Area Name_City Secretary',
 'Business Area Name_Citywide General Government',
 'Business Area Name_Convention and Entertainment',
 'Business Area Name_Finance',
 'Business Area Name_General Debt Services',
 'Business Area Name_General Services Department',
 'Business Area Name_Health & Human Services',
 'Business Area Name_Housing and Community Devp.',
 'Business Area Name_Houston Airport System (HAS)',
 'Business Area Name_Houston Emergency Center (HEC)',
 'Business Area Name_Houston Fire Department (HFD)',
 'Business Area Name_Houston Police Department',
 'Business Area Name_Human Resources Dept.',
 'Business Area Name_Information Technology (IT)',
 'Business Area Name_Legal Department',
 'Business Area Name_Library',
 "Business Area Name_Mayor's Office",
 'Business Area Name_Municipal Court Judicial',
 'Business Area Name_Municipal Courts Department',
 'Business Area Name_Parks & Recreation',
 'Business Area Name_Planning & Development',
 'Business Area Name_Public Works & Engineering-PWE',
 'Business Area Name_Solid Waste Management',
 'Commitment Set Description_50 - Total Expenditures',
 'Business Area Name_Houston Police Department-HPD',
 'Business Area Name_Houston Information Tech Svcs'
]

drop_list = [
    
    'Fund Name', 
    'Fund Type Descr', 
    'Business Area Name',
    'Commitment Set Description',
    'GL Description',
    'GL Category Description',
    'Revenue or Expenditure',
    'Fund Type Descr',
    'Fund Center Name'
]




In [4]:
fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15, fy_16]
del fy_10
del fy_11
del fy_12
del fy_13
del fy_14
del fy_15
del fy_16
gc.collect()

for i, y in enumerate(fy_list):
    fy_list[i] = pd.concat([y[essentials], y[dummies]], axis=1)
    
train_list = [fy_list[2], fy_list[3], fy_list[4]]
test_list = [fy_list[5], fy_list[6]]

X_train = pd.concat(train_list).drop(['OA_Difference','Actuals','Current Budget']+drop_list, axis=1)
X_test = pd.concat(test_list).drop(['OA_Difference', 'Actuals', 'Current Budget']+drop_list, axis = 1)
y_train = pd.concat(train_list)['Actuals']
y_test = pd.concat(test_list)['Actuals']
y_original_budget =  pd.concat(test_list)['Original Budget']

model = xgb.XGBRegressor()
model.fit(X_train,y_train)
print (model)

/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[15:54:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)


In [5]:
xgb_test_predictions = model.predict(X_test)
xgb_train_predictions = model.predict(X_train)
gc.collect()

84

In [6]:
np.sqrt(np.mean((xgb_test_predictions-y_test)**2))

632477.0391946669

In [7]:
model_sum_error = np.sum(abs(xgb_test_predictions-y_test))
model_sum_error

1099110371.7345705

In [8]:
budget_sum_error = np.sum(abs(y_original_budget-y_test))
budget_sum_error

993475373.2600001

### Model precision over budget

In [9]:
print('$',round(budget_sum_error-model_sum_error,2))

$ -105634998.47


In [10]:
np.sqrt(np.mean((y_original_budget-y_test)**2))

519179.9135619907

In [11]:
y_test.std()

6257843.454195455

In [ ]:
sns.set_style('whitegrid')

x = np.linspace(-1000000000,1000000000,1000000000)
plt.figure(figsize=(12,12))
plt.scatter(y_train, xgb_train_predictions, c='red', alpha=.95)
plt.scatter(y_test, xgb_test_predictions, c='blue', alpha=.95)
plt.scatter(y_test, y_original_budget, c='green', alpha=.95)
plt.plot(x,x,c='black')
plt.xlim([-5,1000000000])
plt.ylim([-5,1000000000])
plt.xlabel('observations')
plt.ylabel('predictions')


In [ ]:
weight_dict = model.get_booster().get_score(importance_type="weight")
weight_df = pd.DataFrame(weight_dict, index=range(len(weight_dict)))
weight_df.T[0].sort_values(ascending=False)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test , xgb_test_predictions)

In [ ]:

sns.set_style('darkgrid')
sns.set(font_scale=2.5)
bins = np.arange(0,90000,90000/50)
 
fig, ax = plt.subplots(figsize=(20,15))

sns.distplot(y_test, label='2015 Actuals',color='green', norm_hist=True, bins=bins, kde=False,  ax = ax)
sns.distplot(xgb_test_predictions, label='XGB_test_predictions 2015',color='black', norm_hist=True, bins=bins, kde=True,  ax = ax)
sns.distplot(y_original_budget, label='2015_original_budget',color='blue',norm_hist=True, bins=bins, kde=True,  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(0,90000)
ax.legend()

In [ ]:
original_budget_residuals = fy_15['Original Budget']-fy_15['Actuals']
model_residuals = xgb_test_predictions-y_test
hold_df = pd.DataFrame()
hold_df['Original Budget Residuals'] = original_budget_residuals
hold_df['Model Residuals'] = model_residuals


In [ ]:
np.sqrt(np.mean(model_residuals**2))

In [ ]:
np.sqrt(np.mean(original_budget_residuals**2))

In [ ]:
model_residuals_series = pd.Series(model_residuals)

In [ ]:
original_budget_residuals_series =pd.Series(original_budget_residuals)

In [ ]:
residuals_df = pd.DataFrame()
residuals_df['Model_Residuals'] = model_residuals_series
residuals_df['Original_Budget_Residuals'] = original_budget_residuals_series

extreme_residuals = residuals_df.loc[(residuals_df['Model_Residuals']>=80000) | (residuals_df['Model_Residuals']<=-80000) | (residuals_df['Original_Budget_Residuals']>=80000) | (residuals_df['Original_Budget_Residuals']<=-80000) ] 
residuals_under_80K = residuals_df.loc[(residuals_df['Model_Residuals']<80000) & (residuals_df['Model_Residuals']>-80000) & (residuals_df['Original_Budget_Residuals']<=80000) & (residuals_df['Original_Budget_Residuals']>=-80000) ] 


In [ ]:
print('Model Extreme Resisduals:           ',int(np.sqrt(np.mean(extreme_residuals['Model_Residuals']**2))), '\nOriginal Budget Extreme Resisduals: ',  int(np.sqrt(np.mean(extreme_residuals['Original_Budget_Residuals']**2))))
print(' ')
print('Model Under 80K Resisduals:           ',int(np.sqrt(np.mean(residuals_under_80K['Model_Residuals']**2))), '\nOriginal Budget Under 80K Resisduals: ',  int(np.sqrt(np.mean(residuals_under_80K['Original_Budget_Residuals']**2))))


In [ ]:
sns.set_style('darkgrid')
sns.set(font_scale=2.5)
 
fig, ax = plt.subplots(figsize=(20,15))
bins_ = np.arange(-80000,80000,80000/50)

sns.distplot(model_residuals, label='Model Residuals', norm_hist=True, bins = bins_, color='green',   ax = ax)
sns.distplot(original_budget_residuals, label='Original Budget Residuals',bins=bins_, color='black',  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(-80000,80000)
ax.legend()

# H2o

In [12]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: Picked up _JAVA_OPTIONS: -Xverify:none; java version "12.0.1" 2019-04-16; Java(TM) SE Runtime Environment (build 12.0.1+12); Java HotSpot(TM) 64-Bit Server VM (build 12.0.1+12, mixed mode, sharing)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmppisi9qtu
  JVM stdout: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmppisi9qtu/h2o_josephdown_started_from_python.out
  JVM stderr: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmppisi9qtu/h2o_josephdown_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 10 days
H2O cluster name:,H2O_from_python_josephdown_97mll2
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [13]:
# create h2o frame for training data
training_frame = pd.concat([X_train,y_train], axis=1)
training_frame.columns = list(training_frame.columns[:-1]) + ['Actuals']
training_frame = h2o.H2OFrame(training_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
# create h2o frame for testing data
testing_frame = pd.concat([X_test,y_test], axis=1)
testing_frame.columns = list(testing_frame.columns[:-1]) + ['Actuals']
testing_frame = h2o.H2OFrame(testing_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(None, 'Actuals', training_frame)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [16]:
y_test.std()

6257843.454195455

In [17]:
predicted_actuals = aml.predict(h2o.H2OFrame(pd.concat([X_test, y_test], axis=1)))
np.sqrt(np.mean((predicted_actuals['predict'].as_data_frame()['predict'].values-y_test)**2))

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


815259.9455494743

In [18]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_3_AutoML_20190906_155527

Model Summary: 


,number_of_trees
,278.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 16954271300.2768
RMSE: 130208.5684595173
MAE: 26417.423744033338
RMSLE: NaN
Mean Residual Deviance: 16954271300.2768

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 197714153324.65686
RMSE: 444650.5969012713
MAE: 41725.41228690334
RMSLE: NaN
Mean Residual Deviance: 197714153324.65686
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,41725.293,2326.7915,41477.582,46152.848,42405.65,42626.953,35963.426
mean_residual_deviance,197708268000.0000000,75391877000.0000000,403575472000.0000000,195121709000.0000000,150940484000.0000000,122279051000.0000000,116624597000.0000000
mse,197708268000.0000000,75391877000.0000000,403575472000.0000000,195121709000.0000000,150940484000.0000000,122279051000.0000000,116624597000.0000000
r2,0.9911066,0.0041124,0.9912182,0.9947072,0.9942754,0.9798421,0.9954899
residual_deviance,197708268000.0000000,75391877000.0000000,403575472000.0000000,195121709000.0000000,150940484000.0000000,122279051000.0000000,116624597000.0000000
rmse,431340.0,76335.016,635275.9,441725.8,388510.6,349684.22,341503.44
rmsle,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-09-06 16:45:54,3 min 5.355 sec,0.0,5319990.4984809,280706.2928452,28302298903927.3867188
,2019-09-06 16:45:55,3 min 6.256 sec,5.0,4269006.4529369,224565.8797355,18224416095216.9648438
,2019-09-06 16:45:55,3 min 6.979 sec,10.0,3445233.1936825,183559.9911434,11869631758851.9257812
,2019-09-06 16:45:56,3 min 7.696 sec,15.0,2809080.1565374,151453.5779227,7890931325852.0302734
,2019-09-06 16:45:57,3 min 8.400 sec,20.0,2307466.3120864,126529.7658443,5324400781413.4589844
---,---,---,---,---,---,---
,2019-09-06 16:46:33,3 min 44.106 sec,260.0,137829.1955636,26933.4002410,18996887149.7030678
,2019-09-06 16:46:33,3 min 44.874 sec,265.0,135946.9444943,26809.7110067,18481571717.3258171
,2019-09-06 16:46:34,3 min 45.642 sec,270.0,133658.7375267,26657.3802170,17864658117.2327003
,2019-09-06 16:46:35,3 min 46.397 sec,275.0,131248.1381193,26512.7053369,17226073759.7767258



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
OriginalBudget,4115259817021407232.0000000,1.0,0.5226151
2Y_Past_Actuals,2300734678191046656.0000000,0.5590740,0.2921805
2Y_1Y_Actuals_Mean,769119997220880384.0000000,0.1868946,0.0976740
1Y_Past_Actuals,506744808795013120.0000000,0.1231380,0.0643538
2Y_1Y_Original_Budget_Mean,59524011884806144.0000000,0.0144642,0.0075592
---,---,---,---
FundTypeDescr_EnterpriseFunds,6427578466304.0000000,0.0000016,0.0000008
BusinessAreaName_HoustonAirportSystem(HAS),6407783448576.0000000,0.0000016,0.0000008
FundName_Property&CasualtyFund,4878243463168.0000000,0.0000012,0.0000006
YearlyRainFall,2076443672576.0000000,0.0000005,0.0000003



See the whole table with table.as_data_frame()


In [ ]:
#3:55 pm

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_train, aml.predict(training_frame).as_data_frame()['predict'].values, c='red')
plt.scatter(y_test, aml.predict(testing_frame).as_data_frame()['predict'].values, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')

In [ ]:
#9/4 4:42pm